### Open a coffee shop Model

In [253]:
### import libraries

In [254]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

from bs4 import BeautifulSoup
import csv

!pip install geocoder
import geocoder # to get longitude and latitude


 
import numpy as np
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

print('All libraries and modules installed')

Solving environment: done

# All requested packages already installed.

All libraries and modules installed


<a id='item1'></a>

In [255]:
### Download and Explore the dataset

In [256]:
source = requests.get('https://en.wikipedia.org/wiki/Category:Areas_of_Manchester').text 
soup = BeautifulSoup(source, 'lxml')
csv_file = open('manchester.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Neighbourhood'])

15

In [257]:
mwcg = soup.find_all(class_ = "mw-category-group")

length = len(mwcg) # Gets the length of number of `mw-category-groups` present

for i in range(1, length):  # Gets all the neighbourhoods
    lists = mwcg [i].find_all('a')
    for list in lists:
        nbd = list.get('title') # Gets the title of the neighbourhood
        csv_writer.writerow([nbd]) # Writes the name of the neighbourhood in the csv file

In [258]:
csv_file.close()
manchester_raw = pd.read_csv('manchester.csv')

In [259]:
# Initialize varialbes
lat = []
lng = []
lat_lng_coords = None

# Get postcodes from neighbourhoods table
neighbourhoods = manchester_raw['Neighbourhood']

# Store latitude and longitude values in lat and lng
for nh in neighbourhoods:
    g = geocoder.arcgis('{}, Manchester, UK'.format(nh))
    lat_lng_coords = g.latlng
    lat.append(lat_lng_coords[0])
    lng.append(lat_lng_coords[1])

In [260]:
man_data = manchester_raw
man_data['Latitude'] = lat
man_data['Longitude'] = lng

In [261]:
man_data.head()

,Neighbourhood,Latitude,Longitude
0,Baguley,53.399432,-2.294060
1,Barlow Moor,53.426562,-2.266241
2,"Belle Vue, Manchester",53.463060,-2.197180
3,Benchill,53.389117,-2.265841
4,"Beswick, Manchester",53.483804,-2.215091


### Create a map of venues in Manchester

In [262]:
### we will use foursquare and we will filter only shop venues

In [263]:
CLIENT_ID = 'JW1LOUKCHOBJJGPYYZFROQZINFPN0AU4E44WNXM51NPILF3W' # your Foursquare ID
CLIENT_SECRET = 'QEEXIE4ODGP1UZVI35RD0OQL2XS2WQWE3ZNAQ3T4CMSOMZNJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JW1LOUKCHOBJJGPYYZFROQZINFPN0AU4E44WNXM51NPILF3W
CLIENT_SECRET:QEEXIE4ODGP1UZVI35RD0OQL2XS2WQWE3ZNAQ3T4CMSOMZNJ


### Create a map of Manchester city with neighborhoods imposed on top

In [264]:
from geopy.geocoders import Nominatim

address = 'Manchester, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manchester are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manchester are 53.4794892, -2.2451148.


### Create a map of Manchester city with neighborhoods using latitude and longitude values.

In [265]:
coffee_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(man_data['Latitude'], man_data['Longitude'], man_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(coffee_map)  
    
coffee_map

In [266]:
# Gets the name of the category

def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [267]:
### Function to get all venues in Bronx

In [ ]:
explore_df_list = []

for i, nhood_name in enumerate(man_data['Neighbourhood']):  
    
    try :
        #Get neighbourhood data
        nhood_name = man_data.loc[i, 'Neighbourhood']
        nhood_lat = man_data.loc[i, 'Latitude']
        nhood_lng = man_data.loc[i, 'Longitude']

        radius = 1000 
        LIMIT = 30 

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nhood_lat, nhood_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flatten JSON

        # Filter the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Rename the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Get the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)    

        # Get the required data
        for i, name in enumerate(nearby['Name']):
            s_list = nearby.loc[i, :].values.tolist()  # Converts the numpy array to a python list
            f_list = [nhood_name, nhood_lat, nhood_lng] + s_list
            explore_df_list.append(f_list)
    
    except Exception as e:
        pass

<ipython-input-287-4decbc75657f>:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby = json_normalize(results) # Flatten JSON


### Create a dataframe for clustering

In [ ]:
explore_man = pd.DataFrame([item for item in explore_df_list])
explore_man.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
explore_man.head()

### One-hot encoding to analyze neighbourhoods of Manchester

In [ ]:
man_1hot = pd.get_dummies(explore_man[['Venue Category']], prefix="", prefix_sep="")


# Add neighbourhood column back to dataframe
man_1hot['Neighbourhood'] = explore_man['Neighbourhood'] 


# Move neighbourhood column to the first column
fixed_columns = [man_1hot.columns[-1]] + man_1hot.columns[:-1].values.tolist()
man_1hot = man_1hot[fixed_columns]

man_1hot.head()

#### Sum up the venues in each neighbourhood

In [ ]:
man_grouped = man_1hot.groupby('Neighbourhood').mean().reset_index()
man_grouped.head()

In [ ]:
def top_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

#### Create a new dataframe to get the top 10 venues.


In [ ]:
top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = man_grouped['Neighbourhood']

for ind in np.arange(man_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = top_common_venues(man_grouped.iloc[ind, :], top_venues)

neighbourhoods_venues_sorted.head()

#### Import matplotlib for plotting and apply K-means clustering


In [ ]:
man_grouped_clustering = man_grouped.drop('Neighbourhood', 1)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot(x, y, xlabel, ylabel):
    plt.figure(figsize=(20,10))
    plt.plot(np.arange(2, x), y, 'o-')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(np.arange(2, x))
    plt.show()

In [ ]:
max_range = 15 #Max range 15 (number of clusters)

from sklearn.metrics import silhouette_samples, silhouette_score


indices = []
scores = []

for man_clusters in range(2, max_range) :
    
    # Run k-means clustering
    man_gc = man_grouped_clustering
    kmeans = KMeans(n_clusters = man_clusters, init = 'k-means++', random_state = 0).fit_predict(man_gc)
    
    # Gets the score for the clustering operation performed
    score = silhouette_score(man_gc, kmeans)
    
    # Appending the index and score to the respective lists
    indices.append(man_clusters)
    scores.append(score)

### The optimal number is 5 according to above result


In [ ]:
opt_value = 5

### Proceed with K-Means clustering

In [ ]:
man_clusters = opt_value

# Run k-means clustering
man_gc = man_grouped_clustering
kmeans = KMeans(n_clusters = man_clusters, init = 'k-means++', random_state = 0).fit(man_gc)

In [ ]:
# Add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
man_final = man_data
man_final = man_final.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
man_final.dropna(inplace = True)
man_final['Cluster Labels'] = man_final['Cluster Labels'].astype(int)
man_final.head()

### Folium is used to visualize the clusters

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Setup color scheme for different clusters
x = np.arange(man_clusters)
ys = [i + x + (i*x)**2 for i in range(man_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(man_final['Latitude'], man_final['Longitude'], man_final['Neighbourhood'], 
                                  man_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster + 1) + ')', parse_html=True)
    map_clusters.add_child(
        folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

In [ ]:
val = 1
man_final.loc[man_final['Cluster Labels'] == (val - 1), man_final.columns[[0] + np.arange(4, man_final.shape[1]).tolist()]]

In [ ]:
val = 2
man_final.loc[man_final['Cluster Labels'] == (val - 1), man_final.columns[[0] + np.arange(4, man_final.shape[1]).tolist()]]

In [284]:
val = 3
man_final.loc[man_final['Cluster Labels'] == (val - 1), man_final.columns[[0] + np.arange(4, man_final.shape[1]).tolist()]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baguley,Tram Station,Supermarket,Hardware Store,Donut Shop,Fast Food Restaurant,Gym / Fitness Center,Furniture / Home Store,Coffee Shop,Clothing Store,Pet Store
3,Benchill,Tram Station,Bus Station,Bus Stop,Pub,Supermarket,Bakery,Park,Market,Boat or Ferry,Persian Restaurant
6,Bradford-with-Beswick,Tram Station,Athletics & Sports,Track Stadium,Thai Restaurant,Theater,Canal,Café,Sporting Goods Shop,Park,Sandwich Place
7,"Bradford, Manchester",Tram Station,Athletics & Sports,Track Stadium,Thai Restaurant,Theater,Canal,Café,Sporting Goods Shop,Park,Sandwich Place
19,"Clayton, Manchester",Tram Station,Supermarket,Pub,Discount Store,Bus Stop,Park,Gym / Fitness Center,Grocery Store,Shopping Mall,Coffee Shop
20,Collyhurst,Clothing Store,Coffee Shop,Tram Station,Portuguese Restaurant,Pharmacy,Shopping Plaza,Park,Furniture / Home Store,Fast Food Restaurant,Men's Store
21,Crumpsall,Tram Station,Coffee Shop,Halal Restaurant,Snack Place,Supermarket,Bakery,Pharmacy,Grocery Store,Park,Sandwich Place
37,Miles Platting,Tram Station,Thai Restaurant,Canal,Men's Store,Sandwich Place,Tennis Court,Airport Service,Nature Preserve,Newsagent,Outdoor Supply Store
42,Newall Green,Coffee Shop,Pub,Tram Station,Newsagent,Furniture / Home Store,Sandwich Place,Bus Stop,Convenience Store,Platform,Performing Arts Venue
45,Northern Moor,Grocery Store,Park,Convenience Store,Soccer Field,Lebanese Restaurant,Fast Food Restaurant,Tram Station,Gym / Fitness Center,Electronics Store,Chinese Restaurant


In [285]:
val = 4
man_final.loc[man_final['Cluster Labels'] == (val - 1), man_final.columns[[0] + np.arange(4, man_final.shape[1]).tolist()]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Belle Vue, Manchester",Convenience Store,Supermarket,Gym,Electronics Store,Racetrack,Restaurant,Pharmacy,Fast Food Restaurant,Climbing Gym,Park
9,Burnage,Supermarket,Train Station,Grocery Store,Chinese Restaurant,Coffee Shop,Golf Course,Rental Car Location,Toy / Game Store,Motorcycle Shop,Park
10,Burnage (ward),Supermarket,Train Station,Grocery Store,Chinese Restaurant,Coffee Shop,Golf Course,Rental Car Location,Toy / Game Store,Motorcycle Shop,Park
25,Gorton,Supermarket,Hotel,Movie Theater,Racetrack,Discount Store,Gymnastics Gym,Bakery,Go Kart Track,Train Station,Market
34,Longsight,Indian Restaurant,Supermarket,Park,Market,Sports Club,Turkish Restaurant,College Cafeteria,Electronics Store,Grocery Store,Fast Food Restaurant
41,New Moston,Supermarket,Hotel,Malay Restaurant,Tram Station,Coffee Shop,Train Station,Soccer Stadium,Pizza Place,Pub,Convenience Store
43,Newton Heath,Supermarket,Bus Stop,Athletics & Sports,Gymnastics Gym,Tram Station,Fast Food Restaurant,Park,Shoe Store,Performing Arts Venue,Music Venue
49,Openshaw,Supermarket,Discount Store,Gym / Fitness Center,Pizza Place,Pet Store,Coffee Shop,Farmers Market,Clothing Store,Shopping Mall,Go Kart Track
58,"Victoria Park, Manchester",Pizza Place,Supermarket,Indian Restaurant,Golf Course,Sandwich Place,Chinese Restaurant,Park,Bus Stop,Pub,Hotel
59,West Gorton,Sandwich Place,Discount Store,Train Station,Bus Station,Sports Club,Chinese Restaurant,Climbing Gym,Café,Electronics Store,Convenience Store


In [286]:
val = 5
man_final.loc[man_final['Cluster Labels'] == (val - 1), man_final.columns[[0] + np.arange(4, man_final.shape[1]).tolist()]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Blackley,Hotel,Park,Coffee Shop,Grocery Store,Pet Store,Nature Preserve,Newsagent,Outdoor Supply Store,Pastry Shop,Performing Arts Venue
29,Harpurhey,Hotel,Flea Market,Supermarket,Gym / Fitness Center,Tram Station,Shopping Mall,Park,Pastry Shop,Museum,Music Venue
39,"Moston, Manchester",Hotel,Gym / Fitness Center,Flea Market,Supermarket,Park,Shopping Mall,Business Service,Pie Shop,Pharmacy,Pet Store


### The results show that districts in Ladybarn and Barlow Moor are the best options for opening a new coffee shop with less competition by K-means algorithm. 